## Wishlist

- Adicionar atores no data scraper
- Adicionar leitor de zip

## PG connection - Pegando dados do PG

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PgConnection import ReturningDF

# Define the query to fetch data
query = 'SELECT * FROM public.moviesdb'

# Call the ReturningDF function
ltbd_pg = ReturningDF(query)

# Check the result
if ltbd_pg is not None:
    print(ltbd_pg.head())
else:
    print("Failed to retrieve data.")
    

{'dbname': 'letterboxd', 'user': 'postgres', 'password': '1234', 'host': 'localhost'}
Empty DataFrame
Columns: [Date, Name, Year, Letterboxd_URI, Director, Genres, Themes, Actors]
Index: []


## Datascraper - Adicionando os dados

In [2]:
ltbxd_scp = pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\watched.csv')

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from PgConnection import AddRowToTable

ltbxd_scp.rename(columns={'Letterboxd URI': 'Letterboxd_URI'}, inplace=True)

# Assuming ltbxd_wtchd is your merged DataFrame
for index, row in ltbxd_scp.iterrows():
    title = row['Name']
    
    # Check if the movie already exists in the PostgreSQL DataFrame
    if title in ltbd_pg['Name'].values:
        print(f"Skipping scraping for {title}. Data already exists in PostgreSQL.")
        continue
    
    url = row['Letterboxd_URI']
    
    # Add user-agent headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    # Sending a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # Checking if the request was successful (status code 200)
    if response.status_code == 200:
        # Parsing the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Now you can work with the parsed HTML content
        # For example, you can extract specific information from the webpage
        
        # For demonstration, let's print the title of the webpage
        print(f"Title of webpage {index}: {soup.title.string}")
        
        # Find the span with class 'directorlist'
        director_span = soup.find('span', class_='directorlist')

        # Extract the director's name from the span
        director_name = director_span.text.strip()
        
        # Extract genres from the specific section
        genre_section = soup.find('div', id='tab-genres')
        if genre_section:
            genre_tags = genre_section.find_all('a', class_='text-slug')
            genres = [tag.text for tag in genre_tags if '/films/genre/' in tag['href']]
        else:
            genres = ["Genres not found"]
        
        # Extract themes from the specific section
        if genre_section:
            theme_tags = genre_section.find_all('a', class_='text-slug')
            themes = [tag.text for tag in theme_tags if '/films/theme/' in tag['href'] or '/films/mini-theme/' in tag['href']]
        else:
            themes = ["Themes not found"]
        
        # Extract actors from the specific section
        actor_section = soup.find('div', id='tab-cast')
        if actor_section:
            actor_tags = actor_section.find_all('a', class_='text-slug')
            actors = [tag.text for tag in actor_tags]
        else:
            actors = ["Actors not found"]

             
        # Update the DataFrame with the director, genres, and themes
        ltbxd_scp.at[index, 'Director'] = director_name
        ltbxd_scp.at[index, 'Genres'] = '; '.join(genres)
        ltbxd_scp.at[index, 'Themes'] = '; '.join(themes)
        ltbxd_scp.at[index, 'Actors'] = '; '.join(actors)
        
        # Pass row as a dictionary to AddRowToTable function
        movie_data = tuple(ltbxd_scp.iloc[index])

        # Convert numpy.int64 types to native Python int types
        movie_data = tuple(int(item) if isinstance(item, np.int64) else item for item in movie_data)

        # Call AddRowToTable function with the table name and movie data
        AddRowToTable(movie_data)
    else:
        print(f"Failed to fetch data from URL {url} with status code {response.status_code}")


Title of webpage 0: ‎Barbie (2023) directed by Greta Gerwig • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 1: ‎Come and See (1985) directed by Elem Klimov • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 2: ‎The Sacrifice (1986) directed by Andrei Tarkovsky • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 3: ‎The Fly (1986) directed by David Cronenberg • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 4: ‎Boogie Nights (1997) directed by Paul Thomas Anderson • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 5: ‎La Haine (1995) directed by Mathieu Kassovitz • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 6: ‎Alien (1979) directed by Ridley Scott • Reviews, film + cast • Letterboxd
Movie inserted successfully
Title of webpage 7: ‎Volver (2006) directed by Pedro Almodóvar • Reviews, film + cast • Le

KeyboardInterrupt: 

## PG connection - Adicionando dados de users

In [15]:
from PgConnection import EnteringTable

# Load the CSV file
ratings = pd.read_csv(r'../data/raw/letterboxd-gbmonteiro-2024-05-14-22-51-utc/ratings.csv')

# Add a new column 'User' with the value 'gbmonteiro' for all rows
ratings['User'] = 'gbmonteiro'

# Select only the required columns: 'User', 'Date', 'Rating'
ratings = ratings[['User', 'Date', 'Rating']]

# Insert the DataFrame into the 'public.ratings' table
EnteringTable('public.users', ratings)

ERROR:root:An error occurred: public.users
